In [ ]:
To Do Before Sat:
    
Pull original links to instructional posts
Post on blog?
Run through script and figure out what works, clearly label code that hits the calls per hour limit
Rough outline for what data is available from the api


In [80]:
from datetime import datetime
import json
import keyring
import requests

import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
from pprint import pprint

In [2]:
CLIENT_ID = keyring.get_password("fitbit", "key")
CLIENT_SECRET = keyring.get_password("fitbit", "secret")

server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

[29/Apr/2018:10:49:49] ENGINE Listening for SIGTERM.
[29/Apr/2018:10:49:49] ENGINE Bus STARTING
[29/Apr/2018:10:49:49] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[29/Apr/2018:10:49:49] ENGINE Started monitor thread 'Autoreloader'.
[29/Apr/2018:10:49:49] ENGINE Serving on http://127.0.0.1:8080
[29/Apr/2018:10:49:49] ENGINE Bus STARTED


127.0.0.1 - - [29/Apr/2018:10:49:52] "GET /?code=090300611644a0e2ed5c7fd3d456b8db3446827f&state=F3c3qFd2uvm1ejuSFUs4CE19gQcEc4 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"


[29/Apr/2018:10:49:53] ENGINE Bus STOPPING
[29/Apr/2018:10:49:57] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[29/Apr/2018:10:49:57] ENGINE Stopped thread 'Autoreloader'.
[29/Apr/2018:10:49:57] ENGINE Removed handler for console events.
[29/Apr/2018:10:49:57] ENGINE Bus STOPPED
[29/Apr/2018:10:49:57] ENGINE Bus EXITING
[29/Apr/2018:10:49:57] ENGINE Waiting for child threads to terminate...
[29/Apr/2018:10:49:57] ENGINE Waiting for thread Thread-33.
[29/Apr/2018:10:49:57] ENGINE Bus EXITED


In [ ]:
yesterday2 = str((datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d"))
today = str(datetime.datetime.now().strftime("%Y%m%d"))

In [ ]:
yesterday2 = ((datetime.datetime.now() - datetime.timedelta(days=1)))
yesterday3 = (yesterday2 - datetime.timedelta(days=1))
print(yesterday3)

In [ ]:
import datetime


#take a starting date and a total number of days as an input
# day needs to be in YYYY-MM-DD format
def pullFitBitData(start_date, days, call_type):
    #insert date error checking laterz
    print("Processing day: {}".format(start_date))
    current_date = start_date
    activity_df, heartRate_df = buildActivityData(current_date)
    day_counter = 0
    while day_counter < days:
        current_date = (datetime.datetime.strptime(current_date, "%Y-%m-%d") - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        print("Processing day: {}".format(current_date))
        activity_df2, heartRate_df2 = buildActivityData(current_date)
        activity_df = pd.concat([activity_df, activity_df2])
        heartRate_df = pd.concat([heartRate_df, heartRate_df2])
        day_counter += 1
    print("Ended processing on {}.".format(current_date))
    return activity_df, heartRate_df
        



In [ ]:
active, heart = pullFitBitData('2018-04-22', 30, 'bananas')

In [ ]:
active2, heart2 = pullFitBitData('2018-03-22', 90, 'bananas')

In [ ]:
active3, heart3 = pullFitBitData('2017-12-21', 30, 'bananas')

In [ ]:
current_day = yesterday2

fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=current_day, detail_level='15min')
heartRateZones = (fit_statsHR['activities-heart'][0]['value']['heartRateZones'])

In [ ]:
pprint(heartRateZones)

In [ ]:
#pprint(heartRateZones)

print(heartRateZones_df)

In [ ]:
column_names = heartRatePivot_df.columns.values
new_column_names = []
for name in column_names:
    new_name = name[1].replace(' ', '_')+'.'+name[0]
    new_column_names.append(new_name)
heartRatePivot_df.columns = new_column_names
print(heartRatePivot_df)

In [ ]:
# -within buildActivityData
# -modify url to use requests.get
# -change current_day to date range
# -change processing to loop through results
# -modify results to handle emtpy heartrate results and test
# -modify to take a date when fitbit first entered use




def accumulateHeartData(current_day, heartRateZones):
    date_dict = {'date': [current_day, current_day, current_day, current_day]}
    date_df = pd.DataFrame(date_dict)
    
    heartRateZones_df = pd.DataFrame.from_records(heartRateZones)
    heartRateZones_df['date'] = date_df['date']
    heartRatePivot_df = heartRateZones_df.pivot(index='date', columns='name')
    column_names = heartRatePivot_df.columns.values
    new_column_names = []
    for name in column_names:
        new_name = name[1].replace(' ', '_')+'.'+name[0]
        new_column_names.append(new_name)
    heartRatePivot_df.columns = new_column_names
    return heartRatePivot_df

In [ ]:
def buildActivityData(start_date, end_date):
    activity_stats = auth2_client.activities(date=current_date)['summary']
    heartRateZones = activity_stats['heartRateZones']
    heartRate_df = accumulateHeartData(current_date, heartRateZones)
    del activity_stats['distances']
    del activity_stats['heartRateZones']
    activity_df = pd.DataFrame(activity_stats, index=[current_date])
    return activity_df, heartRate_df

In [36]:

url = "https://api.fitbit.com/1.2/user/-/sleep/date/2018-04-02/2018-04-08.json"
results = requests.get(url = url, headers={'Authorization':'Bearer ' + ACCESS_TOKEN})

In [67]:


sleep_time_events_detail = []
sleep_summaries = []
endpoint = "sleep"

end_date = "2018-04-28"
start_date = "2018-03-29"
status = "continue"

def getEndpointData(endpoint, start_date, end_date):
    url = "https://api.fitbit.com/1.2/user/-/" + endpoint + "/date/" + start_date + "/" + end_date + ".json"
    results = requests.get(url = url, headers={'Authorization':'Bearer ' + ACCESS_TOKEN})
    if results.status_code == 200:
        activity_data = json.loads(results.text)
        return activity_data
    else:
        print(results.text)
        return "ERROR"

while status == "continue":
    activity_data = getEndpointData('sleep', start_date, end_date)
    if activity_data != "ERROR":
        sleep_summaries, sleep_time_events_detail, status = processSleepResults(activity_data)
        start_date, end_date = changeDates(start_date, end_date)
    else:
        break

In [71]:
pprint((sleep_summaries))

[{'dateOfSleep': '2018-04-28',
  'duration': 33960000,
  'efficiency': 97,
  'endTime': '2018-04-28T08:29:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 5,
                                  'minutes': 91,
                                  'thirtyDayAvgMinutes': 57},
                         'light': {'count': 30,
                                   'minutes': 333,
                                   'thirtyDayAvgMinutes': 246},
                         'rem': {'count': 6,
                                 'minutes': 96,
                                 'thirtyDayAvgMinutes': 108},
                         'wake': {'count': 25,
                                  'minutes': 46,
                                  'thirtyDayAvgMinutes': 44}}},
  'logId': 18036487882,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 520,
  'minutesAwake': 46,
  'minutesToFallAsleep': 0,
  'startTime': '2018-04-27T23:03:30.000',
  'timeInBed': 566,
  'type': 'stages'},
 {'dateOfSleep': '2018-04

  'type': 'stages'},
 {'dateOfSleep': '2018-04-07',
  'duration': 28320000,
  'efficiency': 97,
  'endTime': '2018-04-07T09:08:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 4,
                                  'minutes': 76,
                                  'thirtyDayAvgMinutes': 52},
                         'light': {'count': 23,
                                   'minutes': 266,
                                   'thirtyDayAvgMinutes': 242},
                         'rem': {'count': 12,
                                 'minutes': 92,
                                 'thirtyDayAvgMinutes': 116},
                         'wake': {'count': 25,
                                  'minutes': 38,
                                  'thirtyDayAvgMinutes': 41}}},
  'logId': 17811045340,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 434,
  'minutesAwake': 38,
  'minutesToFallAsleep': 0,
  'startTime': '2018-04-07T01:16:30.000',
  'timeInBed': 472,
  'type': 'stages'},
 {'

 {'dateOfSleep': '2018-03-18',
  'duration': 33000000,
  'efficiency': 98,
  'endTime': '2018-03-18T09:00:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 5,
                                  'minutes': 84,
                                  'thirtyDayAvgMinutes': 63},
                         'light': {'count': 22,
                                   'minutes': 265,
                                   'thirtyDayAvgMinutes': 241},
                         'rem': {'count': 7,
                                 'minutes': 171,
                                 'thirtyDayAvgMinutes': 113},
                         'wake': {'count': 17,
                                  'minutes': 30,
                                  'thirtyDayAvgMinutes': 41}}},
  'logId': 17588007006,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 520,
  'minutesAwake': 30,
  'minutesToFallAsleep': 0,
  'startTime': '2018-03-17T23:50:30.000',
  'timeInBed': 550,
  'type': 'stages'},
 {'dateOfSleep': '2018-0

  'minutesToFallAsleep': 0,
  'startTime': '2018-03-09T23:09:00.000',
  'timeInBed': 534,
  'type': 'stages'},
 {'dateOfSleep': '2018-03-09',
  'duration': 3900000,
  'efficiency': 98,
  'endTime': '2018-03-09T16:25:00.000',
  'infoCode': 2,
  'levels': {'summary': {'asleep': {'count': 0, 'minutes': 64},
                         'awake': {'count': 0, 'minutes': 0},
                         'restless': {'count': 1, 'minutes': 1}}},
  'logId': 17494765409,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 64,
  'minutesAwake': 1,
  'minutesToFallAsleep': 0,
  'startTime': '2018-03-09T15:20:00.000',
  'timeInBed': 65,
  'type': 'classic'},
 {'dateOfSleep': '2018-03-09',
  'duration': 29100000,
  'efficiency': 97,
  'endTime': '2018-03-09T06:57:00.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 7,
                                  'minutes': 85,
                                  'thirtyDayAvgMinutes': 59},
                         'light': {'count': 26,
                      

                         'wake': {'count': 21,
                                  'minutes': 35,
                                  'thirtyDayAvgMinutes': 47}}},
  'logId': 17303265844,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 360,
  'minutesAwake': 35,
  'minutesToFallAsleep': 0,
  'startTime': '2018-02-20T00:22:30.000',
  'timeInBed': 395,
  'type': 'stages'},
 {'dateOfSleep': '2018-02-19',
  'duration': 32100000,
  'efficiency': 96,
  'endTime': '2018-02-19T08:58:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 4,
                                  'minutes': 59,
                                  'thirtyDayAvgMinutes': 53},
                         'light': {'count': 31,
                                   'minutes': 287,
                                   'thirtyDayAvgMinutes': 257},
                         'rem': {'count': 8,
                                 'minutes': 136,
                                 'thirtyDayAvgMinutes': 113},
                        

...

                                 'thirtyDayAvgMinutes': 75},
                         'wake': {'count': 21,
                                  'minutes': 48,
                                  'thirtyDayAvgMinutes': 43}}},
  'logId': 15668592079,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 362,
  'minutesAwake': 48,
  'minutesToFallAsleep': 0,
  'startTime': '2017-09-01T00:20:00.000',
  'timeInBed': 410,
  'type': 'stages'},
 {'dateOfSleep': '2017-08-31',
  'duration': 26160000,
  'efficiency': 98,
  'endTime': '2017-08-31T07:56:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 4,
                                  'minutes': 63,
                                  'thirtyDayAvgMinutes': 0},
                         'light': {'count': 29,
                                   'minutes': 255,
                                   'thirtyDayAvgMinutes': 0},
                         'rem': {'count': 3,
                                 'minutes': 75,
                             

                                  'thirtyDayAvgMinutes': 39}}},
  'logId': 15463096737,
  'minutesAfterWakeup': 6,
  'minutesAsleep': 333,
  'minutesAwake': 42,
  'minutesToFallAsleep': 0,
  'startTime': '2017-08-09T00:38:00.000',
  'timeInBed': 375,
  'type': 'stages'},
 {'dateOfSleep': '2017-08-08',
  'duration': 26580000,
  'efficiency': 96,
  'endTime': '2017-08-08T08:08:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 2,
                                  'minutes': 27,
                                  'thirtyDayAvgMinutes': 69},
                         'light': {'count': 24,
                                   'minutes': 260,
                                   'thirtyDayAvgMinutes': 220},
                         'rem': {'count': 6,
                                 'minutes': 103,
                                 'thirtyDayAvgMinutes': 107},
                         'wake': {'count': 25,
                                  'minutes': 53,
                        

                                   'minutes': 196,
                                   'thirtyDayAvgMinutes': 232},
                         'rem': {'count': 3,
                                 'minutes': 85,
                                 'thirtyDayAvgMinutes': 109},
                         'wake': {'count': 17,
                                  'minutes': 31,
                                  'thirtyDayAvgMinutes': 43}}},
  'logId': 15290030095,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 328,
  'minutesAwake': 31,
  'minutesToFallAsleep': 0,
  'startTime': '2017-07-21T00:09:30.000',
  'timeInBed': 359,
  'type': 'stages'},
 {'dateOfSleep': '2017-07-20',
  'duration': 28620000,
  'efficiency': 97,
  'endTime': '2017-07-20T07:04:00.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 2,
                                  'minutes': 40,
                                  'thirtyDayAvgMinutes': 57},
                         'light': {'count': 20,
                         

  'levels': {'summary': {'asleep': {'count': 0, 'minutes': 68},
                         'awake': {'count': 0, 'minutes': 0},
                         'restless': {'count': 0, 'minutes': 0}}},
  'logId': 15099087383,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 68,
  'minutesAwake': 0,
  'minutesToFallAsleep': 0,
  'startTime': '2017-06-30T17:30:00.000',
  'timeInBed': 68,
  'type': 'classic'},
 {'dateOfSleep': '2017-06-30',
  'duration': 26580000,
  'efficiency': 96,
  'endTime': '2017-06-30T07:51:00.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 2,
                                  'minutes': 44,
                                  'thirtyDayAvgMinutes': 51},
                         'light': {'count': 28,
                                   'minutes': 259,
                                   'thirtyDayAvgMinutes': 247},
                         'rem': {'count': 6,
                                 'minutes': 80,
                                 'thirtyDayAvgMinutes': 

  'minutesAwake': 29,
  'minutesToFallAsleep': 0,
  'startTime': '2017-06-16T01:36:30.000',
  'timeInBed': 412,
  'type': 'stages'},
 {'dateOfSleep': '2017-06-15',
  'duration': 27780000,
  'efficiency': 98,
  'endTime': '2017-06-15T08:00:00.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 4,
                                  'minutes': 34,
                                  'thirtyDayAvgMinutes': 54},
                         'light': {'count': 22,
                                   'minutes': 249,
                                   'thirtyDayAvgMinutes': 229},
                         'rem': {'count': 4,
                                 'minutes': 132,
                                 'thirtyDayAvgMinutes': 111},
                         'wake': {'count': 19,
                                  'minutes': 48,
                                  'thirtyDayAvgMinutes': 42}}},
  'logId': 14956170741,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 415,
  'minutesAwake': 48,
  

In [63]:
def processSleepResults(activity_data):

    if not activity_data['sleep']:
        # sleep endpoint no longer returns results
        return sleep_summaries, sleep_time_events_detail, "stop"
    else:
        for sleep_event in activity_data['sleep']:
            sleep_time_events_detail.append(sleep_event['levels']['data'])
            del sleep_event['levels']['data']
            try: 
                del sleep_event['levels']['shortData']
            except:
                pass
                #this was a nap, so no shortData was available
            sleep_summaries.append(sleep_event)
    return sleep_summaries, sleep_time_events_detail, "continue"

In [59]:
pprint(sleep_summaries)

[{'dateOfSleep': '2018-04-28',
  'duration': 33960000,
  'efficiency': 97,
  'endTime': '2018-04-28T08:29:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 5,
                                  'minutes': 91,
                                  'thirtyDayAvgMinutes': 57},
                         'light': {'count': 30,
                                   'minutes': 333,
                                   'thirtyDayAvgMinutes': 246},
                         'rem': {'count': 6,
                                 'minutes': 96,
                                 'thirtyDayAvgMinutes': 108},
                         'wake': {'count': 25,
                                  'minutes': 46,
                                  'thirtyDayAvgMinutes': 44}}},
  'logId': 18036487882,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 520,
  'minutesAwake': 46,
  'minutesToFallAsleep': 0,
  'startTime': '2018-04-27T23:03:30.000',
  'timeInBed': 566,
  'type': 'stages'},
 {'dateOfSleep': '2018-04

                                  'thirtyDayAvgMinutes': 57},
                         'light': {'count': 23,
                                   'minutes': 203,
                                   'thirtyDayAvgMinutes': 244},
                         'rem': {'count': 5,
                                 'minutes': 130,
                                 'thirtyDayAvgMinutes': 120},
                         'wake': {'count': 17,
                                  'minutes': 39,
                                  'thirtyDayAvgMinutes': 40}}},
  'logId': 17768478507,
  'minutesAfterWakeup': 0,
  'minutesAsleep': 404,
  'minutesAwake': 39,
  'minutesToFallAsleep': 0,
  'startTime': '2018-04-03T00:25:30.000',
  'timeInBed': 443,
  'type': 'stages'},
 {'dateOfSleep': '2018-04-02',
  'duration': 25020000,
  'efficiency': 97,
  'endTime': '2018-04-02T07:59:30.000',
  'infoCode': 0,
  'levels': {'summary': {'deep': {'count': 4,
                                  'minutes': 70,
                        

In [35]:
pprint(sleep_data['sleep'][0])
#body_stats = auth2_client.body()   (/sleep/)

#sleep_stats = auth2_client.sleep()

{'awakeCount': 1,
 'awakeDuration': 4,
 'awakeningsCount': 16,
 'dateOfSleep': '2018-04-08',
 'duration': 40020000,
 'efficiency': 97,
 'endTime': '2018-04-08T11:36:00.000',
 'logId': 17821723242,
 'minuteData': [{'dateTime': '00:28:30', 'value': '1'},
                {'dateTime': '00:29:30', 'value': '1'},
                {'dateTime': '00:30:30', 'value': '1'},
                {'dateTime': '00:31:30', 'value': '1'},
                {'dateTime': '00:32:30', 'value': '1'},
                {'dateTime': '00:33:30', 'value': '1'},
                {'dateTime': '00:34:30', 'value': '1'},
                {'dateTime': '00:35:30', 'value': '1'},
                {'dateTime': '00:36:30', 'value': '1'},
                {'dateTime': '00:37:30', 'value': '1'},
                {'dateTime': '00:38:30', 'value': '1'},
                {'dateTime': '00:39:30', 'value': '1'},
                {'dateTime': '00:40:30', 'value': '2'},
                {'dateTime': '00:41:30', 'value': '1'},
                {'d

                {'dateTime': '04:41:30', 'value': '1'},
                {'dateTime': '04:42:30', 'value': '1'},
                {'dateTime': '04:43:30', 'value': '1'},
                {'dateTime': '04:44:30', 'value': '1'},
                {'dateTime': '04:45:30', 'value': '1'},
                {'dateTime': '04:46:30', 'value': '1'},
                {'dateTime': '04:47:30', 'value': '1'},
                {'dateTime': '04:48:30', 'value': '1'},
                {'dateTime': '04:49:30', 'value': '1'},
                {'dateTime': '04:50:30', 'value': '1'},
                {'dateTime': '04:51:30', 'value': '1'},
                {'dateTime': '04:52:30', 'value': '2'},
                {'dateTime': '04:53:30', 'value': '2'},
                {'dateTime': '04:54:30', 'value': '1'},
                {'dateTime': '04:55:30', 'value': '1'},
                {'dateTime': '04:56:30', 'value': '1'},
                {'dateTime': '04:57:30', 'value': '1'},
                {'dateTime': '04:58:30', 'value'

In [ ]:
GET https://api.fitbit.com/1/user/-/body/log/weight/date/[base-date]/[period].json

In [60]:
end_date = "2018-04-28"
start_date = "2018-03-29"

In [65]:
import datetime

def changeDates(start_date, end_date):
    end_date = (datetime.datetime.strptime(end_date, "%Y-%m-%d") - datetime.timedelta(days=30)).strftime("%Y-%m-%d")
    start_date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") - datetime.timedelta(days=30)).strftime("%Y-%m-%d")
    return start_date, end_date

In [74]:
with open('sleep_time_events_detail.txt', 'w') as outfile:
    json.dump(sleep_time_events_detail, outfile)

In [75]:
start_date = "2017-04-28"
end_date = "2018-04-28"
endpoint = "activities/minutesVeryActive"
endpoints = ["activities/calories", "activities/caloriesBMR", "activities/steps", "activities/distance", 
             "activities/floors", "activities/elevation", "activities/minutesSedentary", 
             "activities/minutesLightlyActive", "activities/minutesFairlyActive", "activities/minutesVeryActive",
            "activities/activityCalories"]


data = getEndpointData(endpoint, start_date, end_date)
pprint(test)

{'activities-minutesVeryActive': [{'dateTime': '2017-04-28', 'value': '0'},
                                  {'dateTime': '2017-04-29', 'value': '0'},
                                  {'dateTime': '2017-04-30', 'value': '0'},
                                  {'dateTime': '2017-05-01', 'value': '14'},
                                  {'dateTime': '2017-05-02', 'value': '14'},
                                  {'dateTime': '2017-05-03', 'value': '14'},
                                  {'dateTime': '2017-05-04', 'value': '4'},
                                  {'dateTime': '2017-05-05', 'value': '0'},
                                  {'dateTime': '2017-05-06', 'value': '11'},
                                  {'dateTime': '2017-05-07', 'value': '4'},
                                  {'dateTime': '2017-05-08', 'value': '21'},
                                  {'dateTime': '2017-05-09', 'value': '0'},
                                  {'dateTime': '2017-05-10', 'value': '0'},
       

                                  {'dateTime': '2017-09-10', 'value': '49'},
                                  {'dateTime': '2017-09-11', 'value': '17'},
                                  {'dateTime': '2017-09-12', 'value': '27'},
                                  {'dateTime': '2017-09-13', 'value': '0'},
                                  {'dateTime': '2017-09-14', 'value': '0'},
                                  {'dateTime': '2017-09-15', 'value': '0'},
                                  {'dateTime': '2017-09-16', 'value': '0'},
                                  {'dateTime': '2017-09-17', 'value': '21'},
                                  {'dateTime': '2017-09-18', 'value': '14'},
                                  {'dateTime': '2017-09-19', 'value': '26'},
                                  {'dateTime': '2017-09-20', 'value': '36'},
                                  {'dateTime': '2017-09-21', 'value': '19'},
                                  {'dateTime': '2017-09-22', 'value': '32'},
   

In [78]:
activity_dict_label = endpoint.replace("/","-")
df_label = endpoint.replace("activities/", "")
print(activity_dict_label)
activity_data = test[activity_dict_label]

activity_df = pd.DataFrame(activity_data, columns=['date', df_label ])

activities-minutesVeryActive


In [79]:
print(activity_df)

     date  minutesVeryActive
0     NaN                NaN
1     NaN                NaN
2     NaN                NaN
3     NaN                NaN
4     NaN                NaN
5     NaN                NaN
6     NaN                NaN
7     NaN                NaN
8     NaN                NaN
9     NaN                NaN
10    NaN                NaN
11    NaN                NaN
12    NaN                NaN
13    NaN                NaN
14    NaN                NaN
15    NaN                NaN
16    NaN                NaN
17    NaN                NaN
18    NaN                NaN
19    NaN                NaN
20    NaN                NaN
21    NaN                NaN
22    NaN                NaN
23    NaN                NaN
24    NaN                NaN
25    NaN                NaN
26    NaN                NaN
27    NaN                NaN
28    NaN                NaN
29    NaN                NaN
..    ...                ...
336   NaN                NaN
337   NaN                NaN
338   NaN     